In [1]:
# Imports
from aalpy.learning_algs import run_Lstar
from aalpy.oracles import StatePrefixEqOracle

from DataProcessing import generate_concrete_data_MQTT, split_train_validation
from RNNClassifier import RNNClassifier
from RNN_SULs import Abstract_Mapper_MQTT_RNN_SUL

In [2]:
# Generate concrete MQTT sequances use to train RNN
train_seq, train_labels, input_al, output_al = generate_concrete_data_MQTT(num_examples=300000, num_rand_topics=2,
                                                                               lens=(1, 2, 3, 5, 8, 10, 12),
                                                                               uniform_concretion=True)

# Split it in the training and verification sets
x_train, y_train, x_test, y_test = split_train_validation(train_seq, train_labels, 0.8, uniform=True)

In [3]:
# Create a GRU RNN with 5 layers of size 40
rnn = RNNClassifier(input_al, output_dim=len(output_al), num_layers=5, hidden_dim=40,
                    x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test,
                    batch_size=32, nn_type='GRU')

# Train RNN
rnn.train(epochs=200, stop_acc=1.0, stop_epochs=3)

Starting train
Epoch 0: Accuracy 0.69669, Avg. Loss 4.35797 Validation Accuracy 0.69815
Epoch 1: Accuracy 0.93656, Avg. Loss 0.69118 Validation Accuracy 0.93697
Epoch 2: Accuracy 0.9757, Avg. Loss 0.6257 Validation Accuracy 0.97628
Epoch 3: Accuracy 0.99531, Avg. Loss 0.6318 Validation Accuracy 0.99527
Epoch 4: Accuracy 0.99312, Avg. Loss 0.62972 Validation Accuracy 0.99303
Epoch 5: Accuracy 1.0, Avg. Loss 0.63146 Validation Accuracy 1.0
Epoch 6: Accuracy 1.0, Avg. Loss 0.63073 Validation Accuracy 1.0
Epoch 7: Accuracy 1.0, Avg. Loss 0.60686 Validation Accuracy 1.0
Done training!


In [4]:
# Create the SUL that implements the Mapper component
sul = Abstract_Mapper_MQTT_RNN_SUL(rnn, input_al, output_al)
abstract_inputs = sul.abstract_inputs

# Define the Equivalence Oracle
eq_oracle = StatePrefixEqOracle(abstract_inputs, sul, walks_per_state=50, walk_len=10)

model = run_Lstar(abstract_inputs, sul, eq_oracle, automaton_type='mealy', cache_and_non_det_check=True, print_level=3)

Hypothesis 1: 3 states.
----------------------------------------------------------------------------------------------------------------------------------------
Prefixes / E set                        |('connect',) |('disconnect',) |('subscribe',) |('unsubscribe',) |('publish',)    |('invalid',) 
----------------------------------------------------------------------------------------------------------------------------------------
()                                      |CONNACK      |CONCLOSED       |CONCLOSED      |CONCLOSED        |CONCLOSED       |CONCLOSED    
----------------------------------------------------------------------------------------------------------------------------------------
('connect',)                            |CONCLOSED    |CONCLOSED       |SUBACK         |UNSUBACK         |PUBACK          |CONCLOSED    
----------------------------------------------------------------------------------------------------------------------------------------
('connect', 'subs

In [5]:
print(model)

digraph learnedModel {
s0 [label=s0];
s1 [label=s1];
s2 [label=s2];
s0 -> s1  [label="connect/CONNACK"];
s0 -> s0  [label="disconnect/CONCLOSED"];
s0 -> s0  [label="subscribe/CONCLOSED"];
s0 -> s0  [label="unsubscribe/CONCLOSED"];
s0 -> s0  [label="publish/CONCLOSED"];
s0 -> s0  [label="invalid/CONCLOSED"];
s1 -> s0  [label="connect/CONCLOSED"];
s1 -> s0  [label="disconnect/CONCLOSED"];
s1 -> s2  [label="subscribe/SUBACK"];
s1 -> s1  [label="unsubscribe/UNSUBACK"];
s1 -> s1  [label="publish/PUBACK"];
s1 -> s0  [label="invalid/CONCLOSED"];
s2 -> s0  [label="connect/CONCLOSED"];
s2 -> s0  [label="disconnect/CONCLOSED"];
s2 -> s2  [label="subscribe/SUBACK"];
s2 -> s1  [label="unsubscribe/UNSUBACK"];
s2 -> s2  [label="publish/PUBACK__PUBLISH"];
s2 -> s0  [label="invalid/CONCLOSED"];
__start0 [label="", shape=none];
__start0 -> s0  [label=""];
}

